In [1]:
%%capture
%pip install -e ../../..

In [24]:
from lmexp.models.implementations.gpt2small import GPT2Tokenizer, SteerableGPT2
from lmexp.generic.direction_extraction.probing import train_probe, load_probe
from lmexp.generic.direction_extraction.caa import get_caa_vecs
from lmexp.generic.get_locations import at_search_tokens, all_tokens
from lmexp.generic.activation_steering.steering_approaches import (
    add_multiplier,
)
from lmexp.generic.activation_steering.steerable_model import SteeringConfig
from datetime import datetime
import random

# Load model and tokenizer

These classes have already implemented all the probing-related methods so we won't have to add more hooks + they are ready to use with our vector extraction and steering functions.

In [2]:
model = SteerableGPT2()
tokenizer = GPT2Tokenizer()

In [3]:
model.n_layers

12

# Training a linear probe

## Generate some data

Let's see whether we can get a date/time probe vector

In [5]:
def gen_labeled_text(n):
    # date as text, date as utc timestamp in seconds, sample randomly from between 1990 and 2022
    start_timestamp = datetime(2013, 1, 1).timestamp()
    end_timestamp = datetime(2016, 1, 1).timestamp()
    labeled_text = []
    for i in range(n):
        timestamp = start_timestamp + (end_timestamp - start_timestamp) * random.random()
        date = datetime.fromtimestamp(timestamp)
        # date like "Monday 15th November 2021 8AM"
        text = date.strftime("Today is a %A. It's the %dth of %B, %Y. The time is %I %p. This is the point in time when")
        label = timestamp
        labeled_text.append((text, label))
    # normalize labels to have mean 0 and std 1
    labels = [label for _, label in labeled_text]
    mean = sum(labels) / len(labels)
    std = (sum((label - mean) ** 2 for label in labels) / len(labels)) ** 0.5
    labeled_text = [(text, (label - mean) / std) for text, label in labeled_text]
    return labeled_text

In [6]:
data = gen_labeled_text(10_000)
print(data[0])

("Today is a Monday. It's the 18th of November, 2013. The time is 10 PM. This is the point in time when", -0.7128079377320918)


## Training

In [5]:
# We train a probe with activations extracted from the "when" token
search_tokens = tokenizer.encode("This is the point in time when")[0][-1:]
print(
    f"We train a probe with activations extracted from the '{tokenizer.decode(search_tokens)}' token"
)
save_to = "gpt2small_date_probe.pth"

We train a probe with activations extracted from the ' when' token


In [8]:
probe = train_probe(
    labeled_text=data,
    model=model,
    tokenizer=tokenizer,
    layer=4,
    n_epochs=5,
    batch_size=128,
    lr=1e-2,
    token_location_fn=at_search_tokens,
    search_tokens=search_tokens,
    save_to=save_to,
    loss_type="mse",
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 78/78 [01:28<00:00,  1.14s/it]


Epoch 0, mean loss: 5.2001126152038575


100%|██████████| 78/78 [01:28<00:00,  1.14s/it]


Epoch 1, mean loss: 0.9435745620727539


100%|██████████| 78/78 [01:29<00:00,  1.14s/it]


Epoch 2, mean loss: 0.8537364158630371


100%|██████████| 78/78 [01:28<00:00,  1.14s/it]


Epoch 3, mean loss: 0.7758437637329102


100%|██████████| 78/78 [01:28<00:00,  1.13s/it]

Epoch 4, mean loss: 0.7150826072692871


## Using the vector

In [6]:
probe = load_probe(save_to).to(model.device)
direction = probe.weight[0]
bias = probe.bias

In [7]:
bias

Parameter containing:
tensor([-0.0031], requires_grad=True)

In [9]:
results = model.generate_with_steering(
    text=["The current date is"],
    tokenizer=tokenizer,
    steering_configs=[
        SteeringConfig(
            layer=4,
            vector=direction.detach(),
            scale=-5,
            steering_fn=add_multiplier,
            token_location_fn=all_tokens,
        )
    ],
    max_n_tokens=10,
    save_to=None,
)
print(results['results'])

[{'input': 'The current date is', 'output': 'The current date is the date of the first occurrence'}]


In [10]:
results = model.generate_with_steering(
    text=["The current date is"],
    tokenizer=tokenizer,
    steering_configs=[
        SteeringConfig(
            layer=4,
            vector=direction.detach(),
            scale=10,
            steering_fn=add_multiplier,
            token_location_fn=all_tokens,
        )
    ],
    max_n_tokens=10,
    save_to=None,
)
print(results["results"])

[{'input': 'The current date is', 'output': 'The current date is the next day, so it'}]


# CAA

## Let's get some contrast pairs

Let's try an easy direction - positive vs negative sentiment

In [4]:
GOOD = [
    "The weather is really nice",
    "I'm so happy",
    "This cake is absolutely delicious",
    "I love my friends",
    "I'm feeling great",
    "I'm so excited",
    "This is the best day ever",
    "I really like this gift",
    "Croissants are my favorite",
    "The movie was fantastic",
    "I got a promotion at work",
    "My vacation was amazing",
    "The concert exceeded my expectations",
    "I'm grateful for my family",
    "This book is incredibly engaging",
    "The restaurant service was excellent",
    "I'm proud of my accomplishments",
    "The sunset is breathtakingly beautiful",
    "I passed my exam with flying colors",
    "This coffee tastes perfect",
]

BAD = [
    "The weather is really bad",
    "I'm so sad",
    "This cake is completely inedible",
    "I hate my enemies",
    "I'm feeling awful",
    "I'm so anxious",
    "This is the worst day ever",
    "I dislike this gift",
    "Croissants are disgusting",
    "The movie was terrible",
    "I got fired from work",
    "My vacation was a disaster",
    "The concert was a huge disappointment",
    "I'm frustrated with my family",
    "This book is incredibly boring",
    "The restaurant service was horrible",
    "I'm ashamed of my mistakes",
    "The weather is depressingly gloomy",
    "I failed my exam miserably",
    "This coffee tastes awful",
]

In [5]:
dataset = [
    (text, True) for text in GOOD
] + [
    (text, False) for text in BAD
]

## Getting the CAA vectors

In [6]:
vectors = get_caa_vecs(
    labeled_text=dataset,
    model=model,
    tokenizer=tokenizer,
    layers=range(3, 8),
    token_location_fn=all_tokens,
    save_to=None,
    batch_size=6              
)

100%|██████████| 8/8 [00:00<00:00, 21.75it/s]


## Using the CAA vectors

In [27]:
results = model.generate_with_steering(
    text=["I think that this cat is"],
    tokenizer=tokenizer,
    steering_configs=[
        SteeringConfig(
            layer=5,
            vector=vectors[5],
            scale=-1,
            steering_fn=add_multiplier,
            token_location_fn=all_tokens,
        ),
        SteeringConfig(
            layer=4,
            vector=vectors[4],
            scale=-1,
            steering_fn=add_multiplier,
            token_location_fn=all_tokens,
        ),
    ],
    max_n_tokens=20,
    save_to=None,
)
print(results["results"])

[{'input': 'I think that this cat is', 'output': "I think that this cat is a bit of a liability. I think that it's a liability that"}]


In [37]:
results = model.generate_with_steering(
    text=["I think that this cat is"],
    tokenizer=tokenizer,
    steering_configs=[
        SteeringConfig(
            layer=5,
            vector=vectors[5],
            scale=1,
            steering_fn=add_multiplier,
            token_location_fn=all_tokens,
        ),
        SteeringConfig(
            layer=4,
            vector=vectors[4],
            scale=1,
            steering_fn=add_multiplier,
            token_location_fn=all_tokens,
        ),
    ],
    max_n_tokens=20,
    save_to=None,
)
print(results["results"])

[{'input': 'I think that this cat is', 'output': 'I think that this cat is a great example of how to use the cat as a companion.\n'}]
